In [1]:
import numpy as np
from numpy.linalg import inv
from scipy.special import expit
from scipy.optimize import minimize
import statsmodels.api as sm
from scipy import stats
import datasets

In [2]:
X, y = datasets.htwt()
N, D = X.shape
Xaug = np.hstack([np.ones((N, 1)), X])

In [3]:
model = sm.GLM(y, Xaug, family = sm.families.Binomial()).fit()
np.mean((model.predict(Xaug) > 0.5) != y)

0.11904761904761904

In [14]:
# probit regression
y_tilde = 2 * y - 1

loss = lambda w: Xaug.dot(w).dot(y_tilde)

def grad(w):
    eta = Xaug.dot(w)
    psi_y = stats.norm.pdf(eta) * y_tilde
    Psi = stats.norm.cdf(y_tilde * eta)
    return sum([Xaug[i].dot(psi_y[i]) / Psi[i] for i in xrange(N)])

def hess(w):
    H = np.zeros((D + 1, D + 1))
    eta = Xaug.dot(w)
    for i in xrange(N):
        yeta = y_tilde[i] * eta[i]
        Psi = stats.norm.cdf(yeta)
        psi = stats.norm.pdf(eta[i])
        H -= np.outer(Xaug[i], Xaug[i]) * ((psi / Psi) ** 2 + (yeta * psi / Psi))
    return H

w = minimize(loss, np.zeros(D + 1)).x
# w = minimize(loss, np.zeros(D + 1), method = 'Newton-CG', jac = grad, hess = hess, tol = 1e-6).x
np.mean((Xaug.dot(w) > 0) != y)

0.65238095238095239

In [10]:
# iteratively reweighted least squares -- murphy p. 253
w = np.zeros(D + 1)

for _ in xrange(5):
    eta = Xaug.dot(w)
    mu = expit(eta)
    S = np.diag(mu * (1 - mu))
    z = eta + inv(S).dot(y - mu)
    w = inv(Xaug.T.dot(S).dot(Xaug)).dot(Xaug.T).dot(S).dot(z)

np.mean((expit(Xaug.dot(w)) > 0.5) != y)

0.11904761904761904

In [8]:
def fun(w):
    eta = Xaug.dot(w)
    A = np.log(1 + np.exp(eta))
    return -sum(eta * y - A)
        
w = minimize(fun, np.zeros(D + 1)).x

np.mean((expit(Xaug.dot(w)) > 0.5) != y)

0.11904761904761904